Import Modules

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt   
import os
import cv2
import PIL
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential  

Open Files

In [ ]:
PETS = r'C:\Users\LENOVO\Desktop\Izu\Machine Learning\Deep Learning\PetImages'

Grabbing Files

In [ ]:
import pathlib
PETS = pathlib.Path(PETS)
image_count = len(list(PETS.glob('*/*.jpg')))
image_count

Reading File

In [ ]:
cats = list(PETS.glob('Cat/*.jpg'))
PIL.Image.open(str(cats[0]))

Labeling Process

In [ ]:
Pet_dict = {
    'Dog': list(PETS.glob('Dog/*.jpg')),
    'Cat': list(PETS.glob('Cat/*.jpg'))
}
Pet_labels = {
    'Dog': 1,
    'Cat': 0
}

In [ ]:
img = cv2.imread(str(Pet_dict['Cat'][0]))
# cv2.imshow('img', img)

Seperating Labels and Data

In [ ]:
X, y = [], []
for pet_name, images in Pet_dict.items():
    for image in images:
        if 'jpg' in str(image):
            try:
                img = cv2.imread(str(image))
                resized_img = cv2.resize(img,(50,50))
                X.append(resized_img)
                y.append(Pet_labels[pet_name])
            except Exception as e:
                pass

Transformng Picture to a Numpy Array

In [ ]:
X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
# X_samp = np.random.choice(X, 4000)
# X_samp.shape

Getting Train and Test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=5)

In [ ]:
len(X_test)

Normalizing Data

In [ ]:
X_train = X_train/255
X_test = X_test/255

In [ ]:
X_train

Data Augumentation Layer

In [ ]:
img_height, img_width = 50,50
data_aug = Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

CNN Model

In [ ]:
num_classes = 2
model = Sequential([
    data_aug,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    
    layers.Flatten(), 
    layers.Dense(128, activation='relu'), 
    layers.Dense(num_classes)
])
model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=30)

Model Evaluation

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X, y = [], []
for pet_name, images in Pet_dict.items():
    for image in images:
        if 'jpg' in str(image):
            try:
                img = cv2.imread(str(image))
                resized_img = cv2.resize(img,(224,224))
                X.append(resized_img)
                y.append(Pet_labels[pet_name])
            except Exception as e:
                pass

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=5)

Using Pretrained Model Mobilenet_V2

In [ ]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", input_shape=(224,224,3),
                trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(1, activation='softmax')
])
m.summary()
m.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])
# m.fit(X_train, y_train, epochs=5)

In [ ]:
m.evaluate(X_test, y_test)

Batching Process

In [ ]:
pets_ds = tf.keras.preprocessing.image_dataset_from_directory(
PETS,
shuffle=True,
image_size=(224,224),
batch_size=32
)

In [ ]:
len(pets_ds)

Image Numpy Visualisation

In [ ]:
for file, label in pets_ds.take(1):
    print(file.numpy())
    print(label.numpy())

Split Dataset

In [ ]:
def split_dataset(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 10)
        
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train, val, test = split_dataset(pets_ds)

In [ ]:
len(train),len(val),len(test)

To Multitask CPU and GPU

In [ ]:
# caching,shuffle and prefetching the data
train = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val = val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# num_classes = 2
# model = Sequential([
#     data_aug,
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Dropout(0.2),
    
#     layers.Flatten(), 
#     layers.Dense(128, activation='relu'), 
#     layers.Dense(num_classes)
# ])
# model.compile(optimizer='adam',
#             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#             metrics=['accuracy'])

Batch Model

In [ ]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", input_shape=(224,224,3),
                trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(1, activation='softmax')
])
m.build([32, 224, 224, 3])
m.summary()
m.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            metrics=['accuracy'])
# m.fit(X_train, y_train, epochs=5)

Training Model

In [ ]:
history = m.fit(
    train, 
    epochs = 5, 
    batch_size = 32, 
    verbose = 1, 
    validation_data = val
)